**Let's see how the line if fit to the data in every iteration:**

In [1]:
# Loading 
import pandas as pd

data = pd.read_csv("./data.csv")
temps = data['atemp'].values # in Celsius
rentals = data['cnt'].values /1000

In [2]:
import numpy as np
# Example data (replace with your actual X and y)
X = temps.reshape(-1, 1)
y = rentals
X = np.column_stack((np.ones(len(temps)), X))  # add intercept term

## Line Fit 

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
from tqdm import tqdm

def gradient_descent(X, y, alpha, iterations):
    theta = np.zeros(2)
    m = len(y)
    theta_history = []

    for _ in range(iterations):
        error = X @ theta - y
        theta[0] -= (alpha / m) * np.sum(error)
        theta[1] -= (alpha / m) * np.sum(error * X[:, 1])
        theta_history.append(theta.copy())

    return theta, theta_history

iterations = 2000
alpha = 0.001
theta, theta_history = gradient_descent(X, y, alpha, iterations)

gif_filename = f'gradient_descent_line_lr_{alpha}_iter_{iterations}.gif'
folder = "frames"
os.makedirs(folder, exist_ok=True)
filenames = []

for i in tqdm(range(0, len(theta_history), 5), desc="Generating frames"):
    th = theta_history[i]

    plt.figure(figsize=(6,4))
    plt.scatter(X[:,1], y, color='blue', label='Data')
    plt.plot(X[:,1], X @ th, color='red', label='Regression Line')
    plt.title(f'Iteration {i}')
    plt.xlabel('X')
    plt.ylabel('y')
    plt.legend()

    filename = os.path.join(folder, f'frame_{i}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

with imageio.get_writer(gif_filename, mode='I', duration=0.01) as writer:
    for filename in filenames:
        writer.append_data(imageio.imread(filename))

# Clean up frames
for filename in tqdm(filenames, desc="Cleaning up frames"):
    os.remove(filename)
os.rmdir(folder)

print(f"GIF saved as {gif_filename}")


Generating frames: 100%|██████████| 400/400 [01:42<00:00,  3.91it/s]
C:\Users\HP\AppData\Local\Temp\ipykernel_4208\938244990.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread(filename))
Cleaning up frames: 100%|██████████| 400/400 [00:00<00:00, 2693.66it/s]

GIF saved as gradient_descent_line_lr_0.001_iter_2000.gif


## Gradient Descent Visualization

**Let's see how the gradient descent algorithm work in every iteration:**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
import imageio
import os

# Cost function
def compute_cost(X, y, theta):
    return np.sum((np.matmul(X, theta) - y)**2) / (2 * len(y))

# Gradient descent
def gradient_descent(X, y, alpha, iterations):
    theta = np.zeros(2)
    m = len(y)
    theta_history = []

    for i in range(iterations):
        error = np.dot(X, theta) - y
        t0 = theta[0] - (alpha / m) * np.sum(error)
        t1 = theta[1] - (alpha / m) * np.sum(error * X[:,1])
        theta = np.array([t0, t1])
        theta_history.append(theta.copy())
    
    return theta, theta_history

# Example data
X = temps.reshape(-1, 1)
y = rentals
X = np.column_stack((np.ones(len(temps)), X))  # add intercept term

iterations = 5000
alpha = 0.001
theta, theta_history = gradient_descent(X, y, alpha, iterations)

# Create folder for frames
folder = "frames_3d"
os.makedirs(folder, exist_ok=True)
filenames = []

# Prepare 3D cost surface
theta0_vals = np.linspace(-10, 10, 100)
theta1_vals = np.linspace(-1, 5, 100)
J_vals = np.zeros((len(theta0_vals), len(theta1_vals)))

for i, t0 in enumerate(theta0_vals):
    for j, t1 in enumerate(theta1_vals):
        t = np.array([t0, t1])
        J_vals[i,j] = compute_cost(X, y, t)

theta0_vals, theta1_vals = np.meshgrid(theta0_vals, theta1_vals)

# Plot and save frames
for i, th in enumerate(tqdm(theta_history, desc="Generating 3D frames")):
    if i % 5 != 0:
        continue
    
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(theta0_vals, theta1_vals, J_vals.T, alpha=0.5, cmap='viridis')
    
    # Plot theta path so far
    thetas_so_far = np.array(theta_history[:i+1])
    costs_so_far = np.array([compute_cost(X, y, t) for t in thetas_so_far])
    ax.plot(thetas_so_far[:,0], thetas_so_far[:,1], costs_so_far, color='r', marker='o')
    
    ax.set_xlabel('theta0')
    ax.set_ylabel('theta1')
    ax.set_zlabel('Cost')
    ax.set_title(f'Gradient Descent Iteration {i}')
    
    filename = os.path.join(folder, f'frame_{i}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = f'gradient_descent_3d_lr_{alpha}_iter_{iterations}.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.1) as writer:
    for filename in tqdm(filenames, desc="Creating GIF"):
        image = imageio.imread(filename)
        writer.append_data(image)

# Clean up frames
for filename in tqdm(filenames, desc="Cleaning up frames"):
    os.remove(filename)
os.rmdir(folder)

print(f"3D gradient descent GIF saved as {gif_filename}")

Creating GIF:   0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\HP\AppData\Local\Temp\ipykernel_5948\1339922617.py:81: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
Cleaning up frames: 100%|██████████| 1000/1000 [00:00<00:00, 2445.36it/s]

3D gradient descent GIF saved as gradient_descent_3d_lr_0.001_iter_5000.gif
